In [1]:
import pandas as pd
import urllib.request
import zipfile

import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
path_url = 'https://github.com/mattharrison/datasets/raw/master/data/kaggle-survey-2018.zip'
file_name = 'kaggle-survey-2018.zip'
dataset = 'multipleChoiceResponses.csv'

In [3]:
def extract_dataset(src_path: str, dst_path: str, dataset: str) -> pd.DataFrame: 
    """Extracts a chosen dataset from a zip file and read it into a pandas
    DataFrame.
    
    Parameters:
    ------------
    src_path: str
        URL or path of the zip file to be downloaded.
    dst_path: str
        Local file path where the zip file will be written.
    dataset: str
        Name of the particular file inside the zip file to be 
        read into a DataFrame.
    
    Returns:
    ---------
    pandas.core.frame.DataFrame: 
        DataFrame containing the contents of the selected dataset.
        
    """
    
    # using `urllib.request` module to open the URL specified in `src_path`.    
    open_path = urllib.request.urlopen(src_path)
    
    # reads the contents of the file object `open_path`
    data = open_path.read()
    
    # using context management to write the file in the destination path or folder
    with open(dst_path, mode='wb') as f:
        f.write(data)
       
    # using context management to extract the saved zip file.
    with zipfile.ZipFile(dst_path) as z:
        all_df = pd.read_csv(z.open(dataset))
        df_ = all_df.iloc[0]
        df = all_df.iloc[1:]
        return df

In [4]:
survey_df = extract_dataset(path_url, file_name, dataset)
survey_df.sample(6)

C:\Users\ricky\AppData\Local\Temp\ipykernel_2820\2718579718.py:34: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  all_df = pd.read_csv(z.open(dataset))


,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,...,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,Q50_OTHER_TEXT
9666,4964,Male,-1,22-24,India,Bachelor’s degree,"Computer science (software engineering, etc.)",Student,-1,I am a student,...,-1,NaN,Too time-consuming,Requires too much technical knowledge,Afraid that others will use my work without gi...,Not enough incentives to share my work,NaN,NaN,NaN,-1
3604,467,Female,-1,25-29,United States of America,Master’s degree,"Medical or life sciences (biology, chemistry, ...",Statistician,-1,Academics/Education,...,-1,NaN,NaN,NaN,Afraid that others will use my work without gi...,Not enough incentives to share my work,NaN,NaN,NaN,-1
7309,49085,Male,-1,45-49,South Africa,Bachelor’s degree,Engineering (non-computer focused),Other,349,Other,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,None of these reasons apply to me,NaN,-1
23183,759,Male,-1,25-29,Germany,Master’s degree,Engineering (non-computer focused),Data Scientist,-1,Marketing/CRM,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
7293,1123,Male,-1,40-44,Chile,Doctoral degree,Engineering (non-computer focused),Data Scientist,-1,Broadcasting/Communications,...,-1,Too expensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
4418,507,Male,-1,22-24,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,-1,Insurance/Risk Assessment,...,-1,NaN,Too time-consuming,NaN,NaN,NaN,NaN,NaN,NaN,-1


### Pull off the first two characters of the Q2 column and convert them to an integer.

In [5]:
(survey_df['Q2']
 .str.slice(0,2)
 .astype(int)
)

1        45
2        30
3        30
4        35
5        22
         ..
23855    45
23856    25
23857    22
23858    25
23859    25
Name: Q2, Length: 23859, dtype: int32

### Replace the education strings with numeric values.


In [6]:
(survey_df
 ['Q4']
 .unique()
)

array(['Doctoral degree', 'Bachelor’s degree', 'Master’s degree',
       'Professional degree',
       'Some college/university study without earning a bachelor’s degree',
       'I prefer not to answer', 'No formal education past high school',
       nan], dtype=object)

In [7]:
(survey_df
 ['Q4']
 .replace({'Doctoral degree' : 20, 'Bachelor’s degree' : 16, 'Master’s degree' : 18,
       'Professional degree' : 19,
       'Some college/university study without earning a bachelor’s degree': 13,
       'I prefer not to answer' : None, 'No formal education past high school': 12})
)

1        20.0
2        16.0
3        18.0
4        18.0
5        18.0
         ... 
23855    20.0
23856    18.0
23857    18.0
23858     NaN
23859    18.0
Name: Q4, Length: 23859, dtype: float64

### Take the top three majors, change the others to 'other'.

In [8]:
(survey_df
 ['Q5']
)

1                                                    Other
2                       Engineering (non-computer focused)
3            Computer science (software engineering, etc.)
4        Social sciences (anthropology, psychology, soc...
5                                Mathematics or statistics
                               ...                        
23855        Computer science (software engineering, etc.)
23856                   Engineering (non-computer focused)
23857        Computer science (software engineering, etc.)
23858                                                  NaN
23859                            Mathematics or statistics
Name: Q5, Length: 23859, dtype: object

In [9]:
(survey_df
 ['Q5']
 .value_counts()
)

Computer science (software engineering, etc.)                    9430
Engineering (non-computer focused)                               3705
Mathematics or statistics                                        2950
A business discipline (accounting, economics, finance, etc.)     1791
Physics or astronomy                                             1110
Information technology, networking, or system administration     1029
Medical or life sciences (biology, chemistry, medicine, etc.)     871
Other                                                             770
Social sciences (anthropology, psychology, sociology, etc.)       554
Humanities (history, literature, philosophy, etc.)                269
Environmental science or geology                                  253
I never declared a major                                          128
Fine arts or performing arts                                       87
Name: Q5, dtype: int64

In [10]:
count = (survey_df
 ['Q5']
 .value_counts()
)

(survey_df
 ['Q5']
 .where(survey_df['Q5'].isin(count.index[:4]), 'other')
)

1                                                other
2                   Engineering (non-computer focused)
3        Computer science (software engineering, etc.)
4                                                other
5                            Mathematics or statistics
                             ...                      
23855    Computer science (software engineering, etc.)
23856               Engineering (non-computer focused)
23857    Computer science (software engineering, etc.)
23858                                            other
23859                        Mathematics or statistics
Name: Q5, Length: 23859, dtype: object

### convert to a function

In [11]:
def topx(pser: pd.Series, x: int = 5, default='other') -> pd.Series:
    """Replace all values in a Pandas Series that are not among
    the top `x` most frequent values with a default value.
    
    Parameters
    ----------
    pser : pd.Series
        The input pandas Series.
    x : int, optional
        The number of most frequent values to keep. The
        default value is 5.
    default : str, optional
        The default value to use for values that are not among
        the top `x` most frequent values. The default value is
        'other'.
        
    Returns:
    ---------
    pandas.Series: 
        Series containing the results.
    """
    
    val_count = pser.value_counts()
    return (pser.where(pser
                      .isin(val_count.index[:x]),
                      default)
           )

In [12]:
topx(survey_df['Q5'],)

1                                                other
2                   Engineering (non-computer focused)
3        Computer science (software engineering, etc.)
4                                                other
5                            Mathematics or statistics
                             ...                      
23855    Computer science (software engineering, etc.)
23856               Engineering (non-computer focused)
23857    Computer science (software engineering, etc.)
23858                                            other
23859                        Mathematics or statistics
Name: Q5, Length: 23859, dtype: object

### rename those top three majors to shortened versions.

In [13]:
topx(survey_df['Q5'],).unique()

array(['other', 'Engineering (non-computer focused)',
       'Computer science (software engineering, etc.)',
       'Mathematics or statistics', 'Physics or astronomy',
       'A business discipline (accounting, economics, finance, etc.)'],
      dtype=object)

In [14]:
(topx(survey_df['Q5'],)
 .replace({'Engineering (non-computer focused)': 'eng',
       'Computer science (software engineering, etc.)': 'cs',
       'Mathematics or statistics' : 'stat',})
)

1        other
2          eng
3           cs
4        other
5         stat
         ...  
23855       cs
23856      eng
23857       cs
23858    other
23859     stat
Name: Q5, Length: 23859, dtype: object

### Convert the Q8 column to years of experience 

In [15]:
(survey_df
 ['Q8']
 # .str.replace('+', '', regex=False)
).unique()

array([nan, '5-10', '0-1', '10-15', '3-4', '1-2', '2-3', '15-20', '4-5',
       '20-25', '25-30', '30 +'], dtype=object)

From the above output, these would be my steps:

* replace '+' empty space, 
* split on '-' (the first value of the range) 
* take the left-hand side
* converte that value to a floating point number

In [16]:
(survey_df
 ['Q8']
 .str.replace('+', '', regex=False)
 .str.split('-', expand=True)
 .iloc[:,0]
 .astype(float)
)

1        NaN
2        5.0
3        0.0
4        NaN
5        0.0
        ... 
23855    5.0
23856    NaN
23857    0.0
23858    NaN
23859    NaN
Name: 0, Length: 23859, dtype: float64

### Replace values in the Q9 column 

In [17]:
(survey_df
 ['Q9']
 .unique()
)

array([nan, '10-20,000', '0-10,000',
       'I do not wish to disclose my approximate yearly compensation',
       '20-30,000', '125-150,000', '30-40,000', '50-60,000',
       '100-125,000', '90-100,000', '70-80,000', '80-90,000', '60-70,000',
       '400-500,000', '40-50,000', '150-200,000', '500,000+',
       '300-400,000', '200-250,000', '250-300,000'], dtype=object)

* remove `+` & `,`
* shorten 500,000 to 500, 
* replace the long string with '0'
* split on '-' (the first value of the range) and take the left side, 
* fill in missing values with zero, 
* convert the values to an integer 
* multiply it by 1,000.

In [18]:
(survey_df
 ['Q9']
 .str.replace('+', '', regex=False)
 .str.replace(',','')
 .replace({'500000':'500',
          'I do not wish to disclose my approximate yearly compensation':'0'})
 .str.split('-', expand=True)
 .iloc[:,0]
 .fillna(0)
 .astype(int)
 .mul(1_000)
)

1             0
2         10000
3             0
4             0
5             0
          ...  
23855    250000
23856         0
23857     10000
23858         0
23859         0
Name: 0, Length: 23859, dtype: int32

### Final clean up

In [19]:
(survey_df
 ['Q16_Part_1']
).unique()

array([nan, 'Python'], dtype=object)

* Fill in missing values of `Q16_Part_1` with zero 
* Replace the value `Python` with 1.

In [20]:
(survey_df
 ['Q16_Part_1']
 .fillna(0)
 .replace({'Python': 1})
)

1        0
2        0
3        0
4        1
5        0
        ..
23855    1
23856    0
23857    0
23858    0
23859    0
Name: Q16_Part_1, Length: 23859, dtype: int64

In [21]:
(survey_df
 ['Q16_Part_2']
).unique()

array([nan, 'R'], dtype=object)

* Fill in missing values of `Q16_Part_2` with 0
* Replace the value `R` with 1.

In [22]:
(survey_df
 ['Q16_Part_2']
 .fillna(0)
 .replace({'R': 1})
)

1        0
2        0
3        1
4        1
5        0
        ..
23855    1
23856    0
23857    0
23858    0
23859    0
Name: Q16_Part_2, Length: 23859, dtype: int64

In [23]:
(survey_df
 ['Q16_Part_3']
).unique()

array([nan, 'SQL'], dtype=object)

* Fill in missing values of `Q16_Part_3` with 0
* Replace the value `SQL` with 1.

In [24]:
(survey_df
 ['Q16_Part_3']
 .fillna(0)
 .replace({'SQL': 1})
)

1        0
2        1
3        0
4        1
5        1
        ..
23855    1
23856    0
23857    0
23858    0
23859    0
Name: Q16_Part_3, Length: 23859, dtype: int64

### Create a function
*  rename the columns by replacing spaces with an underscore.
* pull out only the Q1, Q2, age, education, major, years_exp, compensation, python, r, and sql columns.

In [25]:
'Q1,Q3,age,education,major,years_exp,compensation,python,r,sql'.split(',')

['Q1',
 'Q3',
 'age',
 'education',
 'major',
 'years_exp',
 'compensation',
 'python',
 'r',
 'sql']

In [26]:
def prep_data(pdf: pd.DataFrame) -> pd.DataFrame:
    """ This function prepares the rurvey data and returns a new DataFrame.
    The prepatations include extracting and transforming certain
    columns, renaming some, and selecting a subset of columns.
    
    Parameters
    ----------
    pdf : pd.DataFrame
        The input DataFrame containing the survey data.
    
    Returns
    -------
    pd.DataFrame:
        The new DataFrame with the modified and selected columns.
    """
    
    return (pdf
            .assign(age=(pdf['Q2']
                         .str.slice(0,2)
                         .astype(int)
                        ),
                   education=(survey_df
                             ['Q4']
                             .replace({'Doctoral degree' : 20, 'Bachelor’s degree' : 16, 
                                       'Master’s degree' : 18,
                                   'Professional degree' : 19,
                                   'Some college/university study without earning a bachelor’s degree': 13,
                                   'I prefer not to answer' : None, 
                                    'No formal education past high school': 12})
                            ),
                    major=(pdf['Q5']
                           .pipe(topx, x=3)
                           .replace({'Engineering (non-computer focused)': 'eng',
                                       'Computer science (software engineering, etc.)': 'cs',
                                       'Mathematics or statistics' : 'stat',})
                          ),
                    years_exp=(pdf
                             ['Q8']
                             .str.replace('+', '', regex=False)
                             .str.split('-', expand=True)
                             .iloc[:,0]
                             .astype(float)
                            ),
                    compensation=(pdf
                             ['Q9']
                             .str.replace('+', '', regex=False)
                             .str.replace(',','')
                             .replace({'500000':'500',
                                      'I do not wish to disclose my approximate yearly compensation':'0'})
                             .str.split('-', expand=True)
                             .iloc[:,0]
                             .fillna(0)
                             .astype(int)
                             .mul(1_000)
                            ),
                    python=(pdf
                             ['Q16_Part_1']
                             .fillna(0)
                             .replace({'Python': 1})
                            ),
                    r=(pdf
                         ['Q16_Part_2']
                         .fillna(0)
                         .replace({'R': 1})
                        ),
                    sql=(pdf
                         ['Q16_Part_3']
                         .fillna(0)
                         .replace({'SQL': 1})
                        )
                    )
            .rename(columns=lambda col:col.replace(' ', '_'))
            .loc[:, 'Q1,Q3,age,education,major,years_exp,compensation,python,r,sql'.split(',')]
    )

final_df = prep_data(survey_df)
final_df

,Q1,Q3,age,education,major,years_exp,compensation,python,r,sql
1,Female,United States of America,45,20.0,other,NaN,0,0,0,0
2,Male,Indonesia,30,16.0,eng,5.0,10000,0,0,1
3,Female,United States of America,30,18.0,cs,0.0,0,0,1,0
4,Male,United States of America,35,18.0,other,NaN,0,1,1,1
5,Male,India,22,18.0,stat,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
23855,Male,France,45,20.0,cs,5.0,250000,1,1,1
23856,Female,Turkey,25,18.0,eng,NaN,0,0,0,0
23857,Male,Turkey,22,18.0,cs,0.0,10000,0,0,0
23858,Male,United Kingdom of Great Britain and Northern I...,25,NaN,other,NaN,0,0,0,0


The transformer class subclasses the `BaseEstimator` and `TransformerMixin` classes. These
classes require that we implement the `.fit` and `.transform` methods, respectively. The `.fit`
method returns the class instance. The .transform method leverages the logic in the `prep_data`
function

In [27]:
from feature_engine import encoding, imputation
from sklearn import base, pipeline

In [28]:
class PrepDataTransformer(base.BaseEstimator,
    base.TransformerMixin):
    """
    This transformer takes a Pandas DataFrame containing our survey 
    data as input and returns a new version of the DataFrame. 
    
    ----------
    ycol : str, optional
        The name of the column to be used as the target variable.
        If not specified, the target variable will not be set.
    Attributes
    ----------
    ycol : str
        The name of the column to be used as the target variable.
    """
    def __init__(self, ycol=None):
        self.ycol = ycol
    
    def transform(self, X):
        return prep_data(X)

    def fit(self, X, y=None):
        return self


In [29]:
def prepX_y(df, y_col):
    raw = (df
    .query('Q3.isin(["United States of America", "China", "India"]) '
    'and Q6.isin(["Data Scientist", "Software Engineer"])')
    )
    return raw.drop(columns=[y_col]), raw[y_col]

In [30]:
pline = pipeline.Pipeline(
    [('tweak', PrepDataTransformer()),
    ('cat', encoding.OneHotEncoder(top_categories=5, drop_last=True,
    variables=['Q1', 'Q3', 'major'])),
    ('num_impute', imputation.MeanMedianImputer(imputation_method='median',
                variables=['education', 'years_exp']))]
    )


In [31]:
from sklearn import model_selection

In [32]:
X, y = prepX_y(survey_df, 'Q6')

In [33]:
sur_X_train, sur_X_test, sur_y_train, sur_y_test = (model_selection
                                    .train_test_split(X, y, 
                                                      test_size=.3, 
                                                      random_state=42, 
                                                      stratify=y)
                                                        )

In [34]:
X_train = pline.fit_transform(sur_X_train, sur_y_train)
X_test = pline.transform(sur_X_test)
X_train

,age,education,years_exp,compensation,python,r,sql,Q1_Male,Q1_Female,Q1_Prefer not to say,Q1_Prefer to self-describe,Q3_United States of America,Q3_India,Q3_China,major_cs,major_other,major_eng,major_stat
587,25,18.0,4.0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0
3065,22,16.0,1.0,10000,1,0,0,1,0,0,0,0,1,0,1,0,0,0
8435,22,18.0,1.0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
3110,40,20.0,3.0,125000,1,0,1,0,1,0,0,1,0,0,0,1,0,0
16372,45,12.0,5.0,100000,1,0,1,1,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16608,25,16.0,2.0,10000,0,0,1,1,0,0,0,0,1,0,1,0,0,0
7325,18,16.0,1.0,30000,1,0,1,1,0,0,0,1,0,0,1,0,0,0
21810,18,16.0,2.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
4917,25,18.0,1.0,100000,1,0,1,1,0,0,0,1,0,0,0,0,0,1


In [35]:
sur_y_train

587      Software Engineer
3065        Data Scientist
8435        Data Scientist
3110        Data Scientist
16372    Software Engineer
               ...        
16608    Software Engineer
7325     Software Engineer
21810       Data Scientist
4917        Data Scientist
639         Data Scientist
Name: Q6, Length: 2110, dtype: object

Time to create a module I will call in other notebooks as I move along

In [39]:
# %%writefile EffectiveXGBoost.py
# #
# #
# # (c) Ricky Macharm, MScFE
# # https://SisengAI.com
# #
# #

# import pandas as pd
# import urllib.request
# import zipfile

# from feature_engine import encoding, imputation
# from sklearn import base, pipeline

# from sklearn import model_selection

# def extract_dataset(src_path: str, dst_path: str, dataset: str) -> pd.DataFrame: 
#     """Extracts a chosen dataset from a zip file and read it into a pandas
#     DataFrame.
    
#     Parameters:
#     ------------
#     src_path: str
#         URL or path of the zip file to be downloaded.
#     dst_path: str
#         Local file path where the zip file will be written.
#     dataset: str
#         Name of the particular file inside the zip file to be 
#         read into a DataFrame.
    
#     Returns:
#     ---------
#     pandas.core.frame.DataFrame: 
#         DataFrame containing the contents of the selected dataset.
        
#     """
    
#     # using `urllib.request` module to open the URL specified in `src_path`.    
#     open_path = urllib.request.urlopen(src_path)
    
#     # reads the contents of the file object `open_path`
#     data = open_path.read()
    
#     # using context management to write the file in the destination path or folder
#     with open(dst_path, mode='wb') as f:
#         f.write(data)
       
#     # using context management to extract the saved zip file.
#     with zipfile.ZipFile(dst_path) as z:
#         all_df = pd.read_csv(z.open(dataset))
#         df_ = all_df.iloc[0]
#         df = all_df.iloc[1:]
#         return df
    
    

#     def topx(pser: pd.Series, x: int = 5, default='other') -> pd.Series:
#         """Replace all values in a Pandas Series that are not among
#         the top `x` most frequent values with a default value.

#         Parameters
#         ----------
#         pser : pd.Series
#             The input pandas Series.
#         x : int, optional
#             The number of most frequent values to keep. The
#             default value is 5.
#         default : str, optional
#             The default value to use for values that are not among
#             the top `x` most frequent values. The default value is
#             'other'.

#         Returns:
#         ---------
#         pandas.Series: 
#             Series containing the results.
#         """

#         val_count = pser.value_counts()
#         return (pser.where(pser
#                           .isin(val_count.index[:x]),
#                           default)
#                )


# def prep_data(pdf: pd.DataFrame) -> pd.DataFrame:
#     """ This function prepares the rurvey data and returns a new DataFrame.
#     The prepatations include extracting and transforming certain
#     columns, renaming some, and selecting a subset of columns.
    
#     Parameters
#     ----------
#     pdf : pd.DataFrame
#         The input DataFrame containing the survey data.
    
#     Returns
#     -------
#     pd.DataFrame:
#         The new DataFrame with the modified and selected columns.
#     """
    
#     return (pdf
#             .assign(age=(pdf['Q2']
#                          .str.slice(0,2)
#                          .astype(int)
#                         ),
#                    education=(pdf
#                              ['Q4']
#                              .replace({'Doctoral degree' : 20, 'Bachelor’s degree' : 16, 
#                                        'Master’s degree' : 18,
#                                    'Professional degree' : 19,
#                                    'Some college/university study without earning a bachelor’s degree': 13,
#                                    'I prefer not to answer' : None, 
#                                     'No formal education past high school': 12})
#                             ),
#                     major=(pdf['Q5']
#                            .pipe(topx, x=3)
#                            .replace({'Engineering (non-computer focused)': 'eng',
#                                        'Computer science (software engineering, etc.)': 'cs',
#                                        'Mathematics or statistics' : 'stat',})
#                           ),
#                     years_exp=(pdf
#                              ['Q8']
#                              .str.replace('+', '', regex=False)
#                              .str.split('-', expand=True)
#                              .iloc[:,0]
#                              .astype(float)
#                             ),
#                     compensation=(pdf
#                              ['Q9']
#                              .str.replace('+', '', regex=False)
#                              .str.replace(',','')
#                              .replace({'500000':'500',
#                                       'I do not wish to disclose my approximate yearly compensation':'0'})
#                              .str.split('-', expand=True)
#                              .iloc[:,0]
#                              .fillna(0)
#                              .astype(int)
#                              .mul(1_000)
#                             ),
#                     python=(pdf
#                              ['Q16_Part_1']
#                              .fillna(0)
#                              .replace({'Python': 1})
#                             ),
#                     r=(pdf
#                          ['Q16_Part_2']
#                          .fillna(0)
#                          .replace({'R': 1})
#                         ),
#                     sql=(pdf
#                          ['Q16_Part_3']
#                          .fillna(0)
#                          .replace({'SQL': 1})
#                         )
#                     )
#             .rename(columns=lambda col:col.replace(' ', '_'))
#             .loc[:, 'Q1,Q3,age,education,major,years_exp,compensation,python,r,sql'.split(',')]
#     )


# class PrepDataTransformer(base.BaseEstimator,
#     base.TransformerMixin):
#     """
#     This transformer takes a Pandas DataFrame containing our survey 
#     data as input and returns a new version of the DataFrame. 
    
#     ----------
#     ycol : str, optional
#         The name of the column to be used as the target variable.
#         If not specified, the target variable will not be set.
#     Attributes
#     ----------
#     ycol : str
#         The name of the column to be used as the target variable.
#     """
#     def __init__(self, ycol=None):
#         self.ycol = ycol
    
#     def transform(self, X):
#         return prep_data(X)

#     def fit(self, X, y=None):
#         return self

# def prepX_y(df, y_col):
#     raw = (df
#     .query('Q3.isin(["United States of America", "China", "India"]) '
#     'and Q6.isin(["Data Scientist", "Software Engineer"])')
#     )
#     return raw.drop(columns=[y_col]), raw[y_col]